In [26]:
import pandas as pd
from pathlib import Path
import requests
from pprint import pprint
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import re
import math
from scipy.stats import linregress
# Turns off warning messages
import warnings
warnings.filterwarnings("ignore")

In [27]:
# store and read csv file
path = Path("Resources/us_counties_covid19_daily.csv")
covidaily_df = pd.read_csv(path)

In [28]:
# display head
covidaily_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [29]:
# filter & create data frame with my season(Spring)
beg_spring = "2020-02-29"
end_spring = "2020-06-01"
spring_filtered = (covidaily_df["date"] > beg_spring)&(covidaily_df["date"] < end_spring)
covidSP_df = covidaily_df.loc[spring_filtered]
# display covid spring df
covidSP_df

,date,county,state,fips,cases,deaths
398,2020-03-01,Maricopa,Arizona,4013.0,1,0.0
399,2020-03-01,Alameda,California,6001.0,1,0.0
400,2020-03-01,Humboldt,California,6023.0,1,0.0
401,2020-03-01,Los Angeles,California,6037.0,1,0.0
402,2020-03-01,Marin,California,6041.0,1,0.0
...,...,...,...,...,...,...
196845,2020-05-31,Sweetwater,Wyoming,56037.0,30,0.0
196846,2020-05-31,Teton,Wyoming,56039.0,100,1.0
196847,2020-05-31,Uinta,Wyoming,56041.0,12,0.0
196848,2020-05-31,Washakie,Wyoming,56043.0,35,3.0


In [44]:
# finding averages per date, county and state will not be there
spring_avgperdate_df = covidSP_df.groupby(by=["date"]).mean() 
spring_avgperdate_df

,fips,cases,deaths
date,,,
2020-03-01,21051.320000,3.259259,0.111111
2020-03-02,21375.709677,3.151515,0.181818
2020-03-03,21389.181818,3.571429,0.285714
2020-03-04,23706.189189,4.128205,0.307692
2020-03-05,24465.212766,4.653061,0.244898
...,...,...,...
2020-05-27,31443.791074,559.702163,33.755294
2020-05-28,31429.500825,566.150801,34.101342
2020-05-29,31444.735594,572.869125,34.431346


In [45]:
# reset index
spring_avgperdate_df = spring_avgperdate_df.reset_index()

In [46]:
# counting number of counties per date
countiesperday_df =  covidSP_df.groupby(["date"]).count()["county"]
countiesperday_df

date
2020-03-01      27
2020-03-02      33
2020-03-03      35
2020-03-04      39
2020-03-05      49
              ... 
2020-05-27    3052
2020-05-28    3057
2020-05-29    3064
2020-05-30    3066
2020-05-31    3070
Name: county, Length: 92, dtype: int64

In [47]:
#add county to the dateframe?

In [60]:
# Build scatter plot for date vs cases
x_axis = [i for i in range(1,93)]
y_axis = np.array([spring_avgperdate_df["cases"]])
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
regress_vals = x_axis*slope + intercept
line_eqtn = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Incorporate the other graph properties
plt.scatter(x_axis, spring_avgperdate_df["cases"], marker="o", color="blue")
plt.plot(x_axis, regress_vals, "r-")
plt.annotate(line_eqtn, (0,4000), color="red")
plt.title("Days vs Avg COVID-19 Cases")
plt.xlabel("Days")
plt.ylabel("Avg COVID-19 Cases")

# Show plot
plt.show()

TypeError: can't multiply sequence by non-int of type 'numpy.float64'